In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

SQL Queries Used



SEM

SELECT c.date, count(DISTINCT c.click_id), c.experiment_id, c.domain, region, group_index, result_index, click_type, backend, c.device_type, c.subsequent_search, advertiser_domain, c.segment, c.country_code, avg(rpc), DATE_TRUNC('hour',c.time) as TIME


FROM datamart.bu_search.clicks as c 
INNER JOIN datamart.bu_sem.sem_search_account_metadata as ss
ON c.segment = ss.segment
LEFT JOIN datamart.bu_sem.sem_revenue_attribution as r
ON r.click_id = c.click_id


WHERE c.date > '2023-01-01'
AND ss.status ilike 'active'


GROUP BY 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16
ORDER BY c.date, c.experiment_id, TIME



RSOC

SELECT c.date, count(DISTINCT c.click_id), c.experiment_id, c.domain, region, group_index, result_index, click_type, backend, c.device_type, c.subsequent_search, advertiser_domain, c.segment, c.country_code, avg(r.rpc), DATE_TRUNC('hour',c.time) as TIME

FROM datamart.bu_search.clicks as c 
LEFT JOIN datamart.bu_sem.sem_revenue_attribution as r
ON r.click_id = c.click_id


WHERE c.date > '2023-01-01'
AND c.segment LIKE '%c2s%'


GROUP BY 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16
ORDER BY c.date, c.experiment_id, TIME


In [2]:
#reading in rsoc data
rsoc = pd.read_csv('./RSoC_ClicksT.csv')

/var/folders/cn/_nq8fqp91sd6zqky2ymmdj000000gn/T/ipykernel_49596/1521789394.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  rsoc = pd.read_csv('./RSoC_ClicksT.csv')


In [366]:
#reading in sem data
sem = pd.read_csv('./SEM_ClicksT.csv')

/var/folders/cn/_nq8fqp91sd6zqky2ymmdj000000gn/T/ipykernel_49596/3283898437.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sem = pd.read_csv('./SEM_ClicksT.csv')


In [4]:
#cutting down time to just the hour
rsoc['TIME'] = rsoc['TIME'].str[11:19]
sem['TIME'] = sem['TIME'].str[11:19]

In [5]:
#renaming columns for clarity
rsoc.rename(columns = {'COUNT(DISTINCT C.CLICK_ID)':'COUNT', 'AVG(R.RPC)': 'RPC'}, inplace= True)
sem.rename(columns = {'COUNT(DISTINCT C.CLICK_ID)':'COUNT', 'AVG(RPC)': 'RPC'}, inplace= True)

In [6]:
#cleaning up data and filtering data to just be from google and bing
rsoc['DATE'] = pd.to_datetime(rsoc['DATE'])
sem['DATE'] = pd.to_datetime(sem['DATE'])
sem = sem.loc[sem['BACKEND'].isin(['google', 'bing']) ]
rsoc = rsoc.loc[rsoc['BACKEND'].isin(['google', 'bing']) ]
rsoc['DAY'] = rsoc['DATE'].dt.day_name()
sem['DAY'] = sem['DATE'].dt.day_name()

In [365]:
#clicks visualizations

def adclicksovertime(data, name):
    y = data.loc[data['CLICK_TYPE'] == 'ads'].groupby('DATE')['COUNT'].sum().values
    x = data.loc[data['CLICK_TYPE'] == 'ads'].groupby('DATE')['COUNT'].sum().index

    plt.figure(figsize=(20, 5))
    plt.plot(x, y)
    plt.title(f'{name} clicks over time')
    plt.xlabel('Date')
    plt.ylabel('Clicks')
    plt.locator_params(axis = 'x', bins = 15)
    plt.show()

def clicksovertime(data, name):
    y = data.groupby('DATE')['COUNT'].sum().values
    x = data.groupby('DATE')['COUNT'].sum().index

    plt.figure(figsize=(20, 5))
    plt.plot(x, y)
    plt.title(f'{name} clicks over time')
    plt.xlabel('Date')
    plt.ylabel('Clicks')
    plt.locator_params(axis = 'x', bins = 15)
    plt.show()

def clicksbyhour(data, name):
    plt.figure(figsize=(25, 10))
    plt.bar(data.groupby('TIME')['COUNT'].sum().index, data.groupby('TIME')['COUNT'].sum().values) 
    plt.title(f'{name} Clicks by Hour')
    plt.ylabel('Total Clicks')

def initialsearchclickspct(data, name):
    plt.bar(['Initial Search', 'Subsequent Search'], data.groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().values)
    percentage = ((data.groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().values[0]/(data.groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().values[0] + data.groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().values[1])) * 100).round(2)
    print(percentage)
    plt.title(f'{name} Initial Search as Pct of Clicks ' + str(percentage) + '%')
    plt.ylabel('Total Clicks')

def mainlinesidebarclicks(data, name):
    plt.bar(data['REGION'].value_counts().index[:2], data['REGION'].value_counts().values[:2])
    percentage = ((data['REGION'].value_counts().values[0]/ (data['REGION'].value_counts().values[0] + data['REGION'].value_counts().values[1])) * 100).round(2)
    plt.title(f'{name} Mainline Clicks ' + str(percentage) + '%')
    plt.ylabel('Total Clicks')


def clicksbydevice(data, name):
    plt.bar(data.groupby('DEVICE_TYPE')['COUNT'].sum().index, data.groupby('DEVICE_TYPE')['COUNT'].sum().values)
    plt.title(f'{name} Clicks by Device')

def adclicksbydevice(data, name):
    plt.bar(data.loc[data['CLICK_TYPE'] == 'ads'].groupby('DEVICE_TYPE')['COUNT'].sum().index, data.loc[data['CLICK_TYPE'] == 'ads'].groupby('DEVICE_TYPE')['COUNT'].sum().values)
    plt.title(f'{name} Clicks by Device')

def clicksbygroupindexIS(data, name):
    plotdata = pd.DataFrame({

    'Desktop': (data.loc[(data['SUBSEQUENT_SEARCH'] == 0) & (data['GROUP_INDEX'] < 10) & (data['DEVICE_TYPE'] == 'desktop')].groupby('GROUP_INDEX')['COUNT'].sum().values)/sum(data.loc[(data['SUBSEQUENT_SEARCH'] == 0) & (data['GROUP_INDEX'] < 10) & (data['DEVICE_TYPE'] == 'desktop')].groupby('GROUP_INDEX')['COUNT'].sum().values),

    'Mobile': (data.loc[(data['SUBSEQUENT_SEARCH'] == 0) & (data['GROUP_INDEX'] < 10) & (data['DEVICE_TYPE'] == 'mobile')].groupby('GROUP_INDEX')['COUNT'].sum().values)/sum(data.loc[(data['SUBSEQUENT_SEARCH'] == 0) & (data['GROUP_INDEX'] < 10) & (data['DEVICE_TYPE'] == 'mobile')].groupby('GROUP_INDEX')['COUNT'].sum().values)},

    index = data.loc[(data['SUBSEQUENT_SEARCH'] == 0) & (data['GROUP_INDEX'] < 10)].groupby('GROUP_INDEX')['COUNT'].sum().index)

    plotdata.plot(kind="barh")

    plt.title(f'{name} Clicks by Group Index on Initial Search')
    plt.xlabel('Relative Clicks')
    plt.ylabel('Group Index')
    plt.gca().invert_yaxis()
    
def clicksbygroupindexSS(data, name):
    plotdata = pd.DataFrame({

    'Desktop': (data.loc[(data['SUBSEQUENT_SEARCH'] == 1) & (data['GROUP_INDEX'] < 10) & (data['DEVICE_TYPE'] == 'desktop')].groupby('GROUP_INDEX')['COUNT'].sum().values)/sum(data.loc[(data['SUBSEQUENT_SEARCH'] == 1) & (data['GROUP_INDEX'] < 10) & (data['DEVICE_TYPE'] == 'desktop')].groupby('GROUP_INDEX')['COUNT'].sum().values),

    'Mobile': (data.loc[(data['SUBSEQUENT_SEARCH'] == 1) & (data['GROUP_INDEX'] < 10) & (data['DEVICE_TYPE'] == 'mobile')].groupby('GROUP_INDEX')['COUNT'].sum().values)/sum(data.loc[(data['SUBSEQUENT_SEARCH'] == 1) & (data['GROUP_INDEX'] < 10) & (data['DEVICE_TYPE'] == 'mobile')].groupby('GROUP_INDEX')['COUNT'].sum().values)},

    index = data.loc[(data['SUBSEQUENT_SEARCH'] == 1) & (data['GROUP_INDEX'] < 10)].groupby('GROUP_INDEX')['COUNT'].sum().index)

    plotdata.plot(kind="barh")

    plt.title(f'{name} Clicks by Group Index on Subsequent Search')
    plt.xlabel('Relative Clicks')
    plt.ylabel('Group Index')
    plt.gca().invert_yaxis()

def clicksbySS(data, name):
    plotdata = pd.DataFrame({
        'Desktop': data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().values/ sum(data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().values),

        'Mobile': data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().values/ sum(data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().values) },

        index= data.groupby('SUBSEQUENT_SEARCH')['COUNT'].sum().index)

    plotdata.plot(kind="bar")

    plt.title(f'{name} Clicks by SS')
    plt.xlabel('Subsequent Search')
    plt.ylabel('Relative Clicks')

def clicksby0thgroupindex(data, name):
    plotdata = pd.DataFrame({

    'Desktop': (data.loc[(data['GROUP_INDEX'] == 0) & (data['RESULT_INDEX'] < 8) & (data['DEVICE_TYPE'] == 'desktop')].groupby('RESULT_INDEX')['COUNT'].sum().values[:10])/sum(data.loc[(data['GROUP_INDEX'] == 0) & (data['RESULT_INDEX'] < 8) & (data['DEVICE_TYPE'] == 'desktop')].groupby('RESULT_INDEX')['COUNT'].sum().values[:10]),

    'Mobile': (data.loc[(data['GROUP_INDEX'] == 0) & (data['RESULT_INDEX'] < 8) & (data['DEVICE_TYPE'] == 'mobile')].groupby('RESULT_INDEX')['COUNT'].sum().values[:10])/sum(data.loc[(data['GROUP_INDEX'] == 0) & (data['RESULT_INDEX'] < 8) & (data['DEVICE_TYPE'] == 'mobile')].groupby('RESULT_INDEX')['COUNT'].sum().values[:10])},

    index = data.loc[(data['GROUP_INDEX'] == 0) & (data['RESULT_INDEX'] < 8)].groupby('RESULT_INDEX')['COUNT'].sum().index[:10])

    plotdata.plot(kind="barh")
    plt.title(f'{name} Clicks by 0th Group Index')
    plt.gca().invert_yaxis()
    plt.xlabel('Relative Clicks')
    plt.ylabel('Index')

def clicksbydomain(data, name):
    countriesdataind = data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('DOMAIN')['COUNT'].sum().sort_values(ascending = False).index[0:10].tolist()
    countriesdatavalues = data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('DOMAIN')['COUNT'].sum().sort_values(ascending = False).values[0:10].tolist()
    countries = data.groupby('DOMAIN')['COUNT'].mean().sort_values(ascending = False).index[0:10]
    mcountries = []
    for i in countries:
        if i not in countriesdataind:
            mcountries.append(data.loc[(data['DOMAIN'] == i) & (data['DEVICE_TYPE'] == 'mobile')]['COUNT'].sum())
            continue
        valind = countriesdataind.index(i)
        mcountries.append(countriesdatavalues[valind])
    plotdata = pd.DataFrame({

    'Desktop': data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('DOMAIN')['COUNT'].sum().sort_values(ascending = False)[0:10].values,

    'Mobile': mcountries},

    index= data.groupby('DOMAIN')['COUNT'].sum().sort_values(ascending = False)[0:10].index)

    plotdata.plot(kind = 'barh')
    plt.xlabel('Clicks')
    plt.title(f'{name} Clicks by Domain')
    plt.gca().invert_yaxis()

def clicksbyday(data, name):
    daysdataind = data.groupby('DAY')['COUNT'].sum().index.tolist()
    daysdatavalues = data.groupby('DAY')['COUNT'].sum().values.tolist()
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    dayta = []
    for i in days:
        valind = daysdataind.index(i)
        dayta.append(daysdatavalues[valind])
    plt.figure(figsize=(9,7))
    plt.bar(days, dayta)
    plt.ylabel('Clicks')
    plt.title(f'{name} Clicks by Day')

In [362]:
def rpcovertime(data, name):
    y = data.groupby('DATE')['RPC'].mean().values
    x = data.groupby('DATE')['RPC'].mean().index

    plt.figure(figsize=(20, 5))
    plt.plot(x, y)
    plt.title(f'{name} RPC over time')
    plt.xlabel('Date')
    plt.ylabel('RPC')
    plt.locator_params(axis = 'x', bins = 15)
    plt.show()

def rpcbyday(data, name):
    daysdataind = data.groupby('DAY')['RPC'].mean().index.tolist()
    daysdatavalues = data.groupby('DAY')['RPC'].mean().values.tolist()
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    dayta = []
    for i in days:
        valind = daysdataind.index(i)
        dayta.append(daysdatavalues[valind])
    plt.figure(figsize=(9,7))
    plt.bar(days, dayta)
    plt.ylabel('Clicks')
    plt.title(f'{name} Clicks by Day')

def rpcadvertiserbygroupindex(data, backend, name):
    plt.barh(data.loc[(data['GROUP_INDEX'] < 10) & (data['BACKEND'] == backend)].groupby('GROUP_INDEX')['RPC'].mean().index, data.loc[(data['GROUP_INDEX'] < 10) & (data['BACKEND'] == backend)].groupby('GROUP_INDEX')['RPC'].mean().values)
    plt.title(f'{name} Advertiser Avg RPC by Group Index')
    plt.gca().invert_yaxis()
    plt.xlabel('Clicks')
    plt.ylabel('Index')

def rpcbydomain(data, name, device):
    if device == 'mobile':
        plt.barh(data.loc[data['DEVICE_TYPE'] == f'{device}'].groupby('DOMAIN')['RPC'].mean().sort_values(ascending = False).index[0:10], data.loc[data['DEVICE_TYPE'] == f'{device}'].groupby('DOMAIN')['RPC'].mean().sort_values(ascending = False).values[0:10], color = 'darkorange')
    else:
        plt.barh(data.loc[data['DEVICE_TYPE'] == f'{device}'].groupby('DOMAIN')['RPC'].mean().sort_values(ascending = False).index[0:10], data.loc[data['DEVICE_TYPE'] == f'{device}'].groupby('DOMAIN')['RPC'].mean().sort_values(ascending = False).values[0:10])
    plt.title(f'{name} {device} RPC by Domain')
    plt.gca().invert_yaxis()
    plt.xlabel('RPC')
    plt.ylabel('Domain')

def rpcbycountry(data, name):
    countriesdataind = data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('COUNTRY_CODE')['RPC'].mean().sort_values(ascending = False).index[0:10].tolist()
    countriesdatavalues = data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('COUNTRY_CODE')['RPC'].mean().sort_values(ascending = False).values[0:10].tolist()
    countries = data.groupby('COUNTRY_CODE')['RPC'].mean().sort_values(ascending = False).index[0:10]
    mcountries = []
    for i in countries:
        if i not in countriesdataind:
            mcountries.append(data.loc[(data['COUNTRY_CODE'] == i) & (data['DEVICE_TYPE'] == 'mobile')]['RPC'].mean())
            continue
        valind = countriesdataind.index(i)
        mcountries.append(countriesdatavalues[valind])

    plotdata = pd.DataFrame({

    'Desktop': (data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('COUNTRY_CODE')['RPC'].mean().sort_values(ascending = False).values[0:10]),

    'Mobile': mcountries },

    index= data.groupby('COUNTRY_CODE')['RPC'].mean().sort_values(ascending = False).index[0:10])
    plotdata.plot(kind="barh")

    plt.title(f'{name} RPC by Country')
    plt.gca().invert_yaxis()
    plt.xlabel('RPC')
    plt.ylabel('Country Code')

def rpcbycountrydt(data, name, device):
    plotdata = pd.DataFrame({

    f'{device}': (data.loc[data['DEVICE_TYPE'] == f'{device}'].groupby('COUNTRY_CODE')['RPC'].mean().sort_values(ascending = False).values[0:10])},


    index= data.loc[data['DEVICE_TYPE'] == f'{device}'].groupby('COUNTRY_CODE')['RPC'].mean().sort_values(ascending = False).index[0:10])


    plotdata.plot(kind="barh")

    plt.title(f'{name} RPC by Country')
    plt.gca().invert_yaxis()
    plt.xlabel('RPC')
    plt.ylabel('Country Code')

def rpcbysegment(data, name, device):
    plt.barh(data.loc[data['DEVICE_TYPE'] == f'{device}'].groupby('SEGMENT')['RPC'].mean().sort_values(ascending = False).index[0:10], data.loc[data['DEVICE_TYPE'] == f'{device}'].groupby('SEGMENT')['RPC'].mean().sort_values(ascending = False).values[0:10])
    plt.title(f'{name} {device} RPC by Segment')
    plt.gca().invert_yaxis()
    plt.xlabel('RPC')
    plt.ylabel('Segment')

def rpcbyhour(data, name):
    plotdata = pd.DataFrame({

    'Desktop': (data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('TIME')['RPC'].mean().values)/sum(data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('TIME')['RPC'].mean().values),

    'Mobile': (data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('TIME')['RPC'].mean().values)/sum(data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('TIME')['RPC'].mean().values) },

    index= data.groupby('TIME')['RPC'].mean().index)

    plotdata.plot(kind="bar",figsize=(15, 8))

    plt.title("Rpc by Hour")

    plt.xlabel("Time")

    plt.ylabel("RPC")    

def rpcbybackend(data, name):
    plotdata = pd.DataFrame({

    'Desktop': data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('BACKEND')['RPC'].mean().values,

    'Mobile': data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('BACKEND')['RPC'].mean().values },

    index= data.groupby('BACKEND')['RPC'].mean().index)

    plotdata.plot(kind="bar",figsize=(15, 8))
    plt.title(f'{name} RPC by Backend')

def rpcbygeo(data, name):
    plt.bar(data.groupby('COUNTRY_CODE')['RPC'].mean().index, data.groupby('BACKEND')['RPC'].mean().values)
    plotdata = pd.DataFrame({

    'Desktop': data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('COUNTRY')['RPC'].mean().values,

    'Mobile': data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('COUNTRY')['RPC'].mean().values},

    index= data.groupby('COUNTRY')['RPC'].mean().index)

    plotdata.plot(kind="barh")
    plt.title(f'{name} RPC by Geo')

def rpcbydevice(data, name):
    plt.bar(data.groupby('DEVICE_TYPE')['RPC'].mean().index, data.groupby('DEVICE_TYPE')['RPC'].mean().values)
    plt.title(f'{name} RPC by Device')

def rpcbysearch(data, name):
    plotdata = pd.DataFrame({

    'Desktop': data.loc[data['DEVICE_TYPE'] == 'desktop'].groupby('SUBSEQUENT_SEARCH')['RPC'].mean().values,

    'Mobile': data.loc[data['DEVICE_TYPE'] == 'mobile'].groupby('SUBSEQUENT_SEARCH')['RPC'].mean().values },

    index= data.groupby('SUBSEQUENT_SEARCH')['RPC'].mean().index)

    plotdata.plot(kind="bar")
    plt.title(f'{name} RPC by Subsequent Search')


def rpcbyadvertiser(data, name):
    plt.bar(data.groupby('ADVERTISER_DOMAIN')['RPC'].mean().sort_values(ascending = False).index[0:5], data.groupby('ADVERTISER_DOMAIN')['RPC'].mean().sort_values(ascending = False).values[0:5])
    plt.title(f'{name} RPC by Advertiser')
    plt.xticks(rotation = 45)

def rpcbygroup0index(data,  name):
    plt.barh(data.loc[(data['GROUP_INDEX'] == 0)].groupby('RESULT_INDEX')['RPC'].mean().index[:10], data.loc[(data['GROUP_INDEX'] == 0)].groupby('RESULT_INDEX')['RPC'].mean().values[:10])
    plt.title(f'{name} Avg RPC by Group 0 Result Index')
    plt.gca().invert_yaxis()
    plt.xlabel('RPC')
    plt.ylabel('Index')